# Explore GCOOS Terrain COGs
This 1 arc second (~30m) resolution terrain data as 32-bit cloud-optimized geotiff in a public AWS bucket.  Each file is a 1x1 degree square. 

In [ ]:
import hvplot.xarray
import fsspec
import os
import rioxarray

In [ ]:
os.environ['GDAL_DISABLE_READDIR_ON_OPEN']='EMPTY_DIR' #This is KEY! otherwise we send a bunch of HTTP GET requests to test for common sidecar metadata
os.environ['AWS_NO_SIGN_REQUEST']='YES' #Since this is a public bucket, we don't need authentication
os.environ['GDAL_MAX_RAW_BLOCK_CACHE_SIZE']='200000000'  #200MB: Want this to be greater than size of uncompressed raster to overcome a 10 MB limit in the GeoTIFF driver for range request merging.
os.environ['GDAL_SWATH_SIZE']='200000000'  #also increase this if increasing MAX_RAW_BLOCK_CACHE_SIZE
os.environ['VSI_CURL_CACHE_SIZE']='200000000' #also increase this if increasing MAX_RAW_BLOCK_CACHE_SIZE

In [ ]:
import sys
sys.path.append(os.path.join(os.environ['HOME'],'shared','users','lib'))
import ebdpy as ebd

ebd.set_credentials(profile='esip-qhub')

profile = 'esip-qhub'
region = 'us-west-2'
endpoint = f's3.{region}.amazonaws.com'
ebd.set_credentials(profile=profile, region=region, endpoint=endpoint)
worker_max = 10
client,cluster = ebd.start_dask_cluster(profile=profile,worker_max=worker_max, 
                                      region=region, use_existing_cluster=True,
                                      adaptive_scaling=False, wait_for_cluster=False, 
                                      environment='pangeo', worker_profile='Pangeo Worker', 
                                      propagate_env=True)

In [ ]:
#client.close();cluster.shutdown()   # shutdown client and cluster

Use GDAL settings from [COG Best Practices](https://github.com/pangeo-data/cog-best-practices/blob/main/0-single-cog.ipynb)

In [ ]:
fs = fsspec.filesystem('s3', anon=True)

In [ ]:
fs.ls('coastalcoupling/testing/')

In [ ]:
s3url = 's3://coastalcoupling/testing/GCOOS_bathymetry_cog.tif'

In [ ]:
da = rioxarray.open_rasterio(s3url, masked=True, overview_level=0,
                            chunks={'x':2048,'y':2048}).squeeze(dim='band', drop=True)

In [ ]:
da

Mask out the ocean

In [ ]:
da = da.where(da<1e16)

Visualize with hvplot from the holoviz.org tool suite

In [ ]:
da.hvplot.image(x='x',y='y', geo=True, rasterize=True, cmap='turbo', tiles='OSM', alpha=0.7, frame_width=700)